In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [53]:
#Loading data
df = pd.read_csv("Data/insurance - insurance.csv")
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


# 3 Question to Answer:

- Q1. Do smokers have higher insurance charges than non-smokers?
- Q2. Are men more likely to smoke than women?
- Q3. Do different regions have different charges, on average?

In [54]:
df["sex"].value_counts()

male      676
female    662
Name: sex, dtype: int64

# Non Smokers vs Smokers Insurance Charges

1.State your Null Hypothesis and Alternative Hypothesis

   * Null Hypothesis: There is no difference in insurance charges with non-smokers and smokers.
    
   * Alternative Hypothesis: There is a significant difference in insurance charges between non-smokers and smokers.

2.Select the correct test according to the data type and number of samples

   * 2 Sample T-Test (independent)

3.Test the assumptions of your selected test.

   *  No significant outliers
   *  Normality
   *  Equal Variance


4.Execute the selected test, or the alternative test (if you do not meet the assumptions)
   
   * I'll be looking out for the p-value from the .levene variance test to determine whether to use `equal_var = False` when running the T-Test.

5.Interpret your p-value and reject or fail to reject your null hypothesis 
 
    
6.Show a supporting visualization that helps display the result


## Testing for Outliers



In [55]:
#Checking the number of smokers to see if our sample is enough
df["smoker"].value_counts()

no     1064
yes     274
Name: smoker, dtype: int64

In [56]:
#Splitting up data between smokers and non smokers
non_smoker = df.loc[df["smoker"]=="no"]
smoker = df.loc[df["smoker"]=="yes"]

In [57]:
#Define our feature of interest
non_smoker_charges = non_smoker["charges"]
smoker_charges = smoker["charges"]

## Check for and remove outliers

In [58]:
# Check for outliers in charges for non-smoker group
zscores = stats.zscore(non_smoker_charges)
outliers = abs(zscores)>3
np.sum(outliers)

24

In [59]:
# remove outliers from non-smoker group
non_smoker_charges = non_smoker_charges[(np.abs(stats.zscore(non_smoker_charges)) < 3)]

In [60]:
# Check for outliers in charges for smoker group
zscores = stats.zscore(smoker_charges)
outliers = abs(zscores)>3
np.sum(outliers)

0

In [61]:
# remove outliers from smoker group
smoker_charges = smoker_charges[(np.abs(stats.zscore(smoker_charges)) < 3)]

## Check for Normality



In [62]:
#Checking for normality in non-smokers
result_non_smokers = stats.normaltest(non_smoker_charges)
result_non_smokers

NormaltestResult(statistic=163.80367047789198, pvalue=2.6945416315543976e-36)

pvalue for non-smokers is > .05, meaning that it is normal distribution

In [63]:
#checking for normalitiy in smokers
result_smokers = stats.normaltest(smoker_charges)
result_smokers

NormaltestResult(statistic=61.03941356533816, pvalue=5.564930630036463e-14)

pvalue for smokers is < .05, meaning that it is NOT a normal distribution. However, our sample size is large enough to proceed without satisfying this test.  For an independent t-test, sample sizes of 15 per group are considered sufficient.



## Check for Equal Variance

In [65]:
# Test for equal variance
result = stats.levene(non_smoker_charges, smoker_charges)
result

LeveneResult(statistic=520.7468821724297, pvalue=2.4247238784347824e-97)

Smokers vs. Non-smokers doesnt have equal variance (pvalue > .05) but I will use `equal_var = False` when we perform our t-test.

## Perform and evaluate the t-test

In [67]:
# Independent t-test with equal_var set to False
result = stats.ttest_ind(smoker_charges, non_smoker_charges, equal_var = False)
result

Ttest_indResult(statistic=33.732305987092516, pvalue=2.5753226625873578e-104)

The p-value < .05, so i reject the null hypothesis and find significance in charges between smokers and non-smokers.